### 任务二、使用MTCNN实现人脸跟踪

本案例使用 `facenet_pytorch` Python软件包MTCNN在图像数据集上执行人脸检测和跟踪。

### 0. 环境准备

In [1]:
!pip install -U openmim
!mim install mmcv

Looking in indexes: https://pypi.virtaicloud.com/repository/pypi/simple
Looking in indexes: https://pypi.virtaicloud.com/repository/pypi/simple
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html


In [2]:
!pip install facenet_pytorch==2.5.2

Looking in indexes: https://pypi.virtaicloud.com/repository/pypi/simple


导入依赖包

In [3]:
from facenet_pytorch import MTCNN
import torch
import numpy as np
import mmcv, cv2
from PIL import Image, ImageDraw
from IPython import display

/root/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


根据当前运行环境是否有GPU可用，设置device的值

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('在该设备上运行: {}'.format(device))

在该设备上运行: cuda:0


#### 1. 定义MTCNN模块


<font color=red size=3>动手练习1</font> <br>

在`<1>`处，创建MTCNN的一个实例，这里需要设置保留所有检测到的人脸，设置运行设备为上一步中检测出的设备，其它参数可保持默认值；

具体设置的参数可以通过`help(MTCNN)`查看文档。

In [5]:
mtcnn = MTCNN(keep_all=True, device=device)

In [6]:
!cp $GEMINI_DATA_IN1/video.mp4 $GEMINI_CODE

#### 2. 读取视频帧

此步需要读取视频中的每一帧，并将帧转换为PIL图像。

<font color=red size=3>动手练习2</font> <br>

在`<1>`处，写一个表达式，将每一帧`frame`，使用cv2.cvtColor函数将其从OpenCV使用的BGR颜色空间转换为RGB颜色空间，然后使用Image.fromarray将转换后的帧转换为PIL图像。

执行以下代码，可以播放视频。

In [7]:
video = mmcv.VideoReader('video.mp4')
frames = [Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)) for frame in video]

display.Video('video.mp4', width=640)

#### 3. 迭代视频帧，并绘制人脸框

<font color=red size=3>动手练习3</font> <br>

在`<1>`处，使用 mtcnn 对当前帧 frame 进行人脸检测，返回检测到的所有人脸的边界框坐标 boxes。

在`<2>`处，使用`draw.rectangle()`方法绘制box矩形框，框的颜色为红色 (255, 0, 0)，宽度为6。

In [8]:
frames_tracked = []
for i, frame in enumerate(frames):
    print('\r当前正在处理帧: {}'.format(i + 1), end='')
    
    # 检测人脸
    boxes, _ = mtcnn.detect(frame)
    
    # 绘制人脸框
    frame_draw = frame.copy()
    draw = ImageDraw.Draw(frame_draw)
    for box in boxes:
        draw.rectangle(box.tolist(), outline=(255, 0, 0), width=6)
    
    # 添加到图像列表
    frames_tracked.append(frame_draw.resize((640, 360), Image.BILINEAR))
print('\n结束')

当前正在处理帧: 105
结束


#### 显示人脸帧

<font color=red size=3>动手练习4</font> <br>

<1>处补充代码，写一个循环展示上一步获取的`frames_tracked`中保存的人脸帧。

In [ ]:
d = display.display(frames_tracked[0], display_id=True)
i = 1
try:
    while i < 106:
        d.update(frames_tracked[i % len(frames_tracked)])
        i += 1
except KeyboardInterrupt:
    pass

##### 保存为视频

执行以下代码，可人脸帧保存为视频。

In [ ]:
dim = frames_tracked[0].size
fourcc = cv2.VideoWriter_fourcc(*'FMP4')    
video_tracked = cv2.VideoWriter('video_tracked.mp4', fourcc, 25.0, dim)
for frame in frames_tracked:
    video_tracked.write(cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR))
video_tracked.release()